In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from google.colab import drive
drive.mount('/content/drive')

###Loading Data

In [13]:
data = pd.read_csv('/content/drive/MyDrive/Retinopathy Dataset')
X = data[list(data.columns)[:-1]]
y = data[list(data.columns)[-1]]

In [26]:
X.head()

,age,systolic_blood_pressure,diastolic_blood_pressure,cholesterol
0,77.196340,85.288742,80.021878,79.957109
1,63.529850,99.379736,84.852361,110.382411
2,69.003986,111.349455,109.850616,100.828246
3,82.638210,95.056128,79.666851,87.066303
4,78.346286,109.154591,90.713220,92.511770


In [15]:
y.head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: has_retinopathy, dtype: float64

In [19]:
X.shape

(6000, 4)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

In [23]:
X_train.head()

,age,systolic_blood_pressure,diastolic_blood_pressure,cholesterol
3381,75.545442,113.847822,109.234402,105.807120
31,63.330546,84.724319,79.364044,98.989098
1596,52.581121,103.514148,93.467931,95.064684
1386,57.265193,94.820458,79.113362,88.962520
4237,50.142801,93.779273,88.373771,89.746829


###Implementing C-Index

In [35]:
def cindex(y_true, scores):
  # y_true: a 1-D array of true binary outcomes (values of zero or one)
  # scores: a 1-D array of corresponding risk scores output by the model
  # c_index: (concordant pairs + 0.5*ties) / number of permissible pairs
  assert len(scores) == len(scores)
  n = len(y_true)


  concordant = 0
  permissible = 0
  ties = 0

  for i in range(n):
    for j in range(i+1, n):
      # Check if the pair is permissible (the patient outcomes are different)
      if y_true[i] != y_true[j]:
        permissible += 1

      # For permissible pairs, check if they are concordant or are ties

      # case 1: checking for ties
      if scores[i] == scores[j]:
        ties += 1
        continue

      # case 2: patient i doesn't get the disease, patient j does
      if y_true[i] == 0 and y_true[j] == 1:
        # Check if patient i has a lower risk score than patient j
        if scores[i] < scores[j]:
          concordant += 1


      # case 3: patient i gets the disease, patient j does not
      if y_true[i] == 1 and y_true[j] == 0:
      # Check if patient i has a higher risk score than patient j
        if scores[i] > scores[j]:
          concordant += 1

  c_index = (concordant + 0.5 * ties) / permissible
  return c_index

###Experiment-1: Normalize data and evaluate

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [31]:
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

LogisticRegression()

In [33]:
# Accuracy
y_pred = model.predict(X_test_scaled)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.77      0.74      0.75       594
         1.0       0.75      0.78      0.77       606

    accuracy                           0.76      1200
   macro avg       0.76      0.76      0.76      1200
weighted avg       0.76      0.76      0.76      1200



In [46]:
# C-index
scores = model.predict_proba(X_test_scaled)[:, 1]
c_index_X_test = cindex(y_test.values, scores)
print(f"c-index on test set is {c_index_X_test:.4f}")

c-index on test set is 0.8242


In [62]:
for i in range(len(X_train.columns)):
    print(f"{X_train.columns[i]}: {model.coef_[0][i]:.2f}")

age: 1.12
systolic_blood_pressure: 0.73
diastolic_blood_pressure: 0.02
cholesterol: 0.62


###Experiment 2: Adding Interaction variable

In [69]:
def add_interactions(X):
  # X: dataframe

  features = X.columns
  m = len(features)
  X_int = X.copy(deep=True)

  for i in range(m):
    feature_i_name = features[i]
    feature_i_data = X.loc[:, feature_i_name]

    for j in range(i+1, m):
      feature_j_name = features[j]
      feature_j_data = X.loc[:, feature_j_name]
      feature_i_j_name = f"{feature_i_name}_x_{feature_j_name}"
      X_int[feature_i_j_name] = feature_i_data * feature_j_data

  return X_int

In [70]:
X_train_int = add_interactions(X_train)
X_test_int = add_interactions(X_test)

In [93]:
# Normalize
scaler_int = StandardScaler()

X_train_int_scaled = scaler_int.fit_transform(X_train_int)
X_test_int_scaled = scaler_int.transform(X_test_int)

In [94]:
model_int = LogisticRegression()
model_int.fit(X_train_int_scaled, y_train)

LogisticRegression()

In [95]:
for i in range(len(X_train_int.columns)):
  print(f"{X_train_int.columns[i]}: {model_int.coef_[0][i]:.2f}")

age: 3.81
systolic_blood_pressure: -0.09
diastolic_blood_pressure: 0.26
cholesterol: 2.22
age_x_systolic_blood_pressure: 0.77
age_x_diastolic_blood_pressure: -0.49
age_x_cholesterol: -3.58
systolic_blood_pressure_x_diastolic_blood_pressure: -0.06
systolic_blood_pressure_x_cholesterol: 0.72
diastolic_blood_pressure_x_cholesterol: 0.14


In [101]:
X_train_int_scaled_df = pd.DataFrame(X_train_int_scaled, columns=X_train_int.columns)
X_test_int_scaled_df = pd.DataFrame(X_test_int_scaled, columns=X_train_int.columns)

In [108]:
# including age_x_cholesterol, age, sys_bp and cholesterol in model
X_train_imp = X_train_int_scaled_df[['age', 'age_x_systolic_blood_pressure', 'cholesterol','age_x_cholesterol']]
X_test_imp = X_test_int_scaled_df[['age', 'age_x_systolic_blood_pressure', 'cholesterol','age_x_cholesterol']]

In [109]:
model_imp = LogisticRegression()
model_imp.fit(X_train_imp, y_train)

LogisticRegression()

In [110]:
# Accuracy
y_pred = model_imp.predict(X_test_imp)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.78      0.74      0.76       594
         1.0       0.76      0.80      0.78       606

    accuracy                           0.77      1200
   macro avg       0.77      0.77      0.77      1200
weighted avg       0.77      0.77      0.77      1200



In [111]:
# C-index
scores = model_imp.predict_proba(X_test_imp)[:, 1]
c_index = cindex(y_test.values, scores)
print(f"c-index on test set is {c_index:.4f}")

c-index on test set is 0.8297
